In [4]:
from zipfile import ZipFile
import pandas as pd
import kaggle
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

In [3]:
# https://www.kaggle.com/kimjihoo/coronavirusdataset
kaggle.api.authenticate()
kg_url='kimjihoo/coronavirusdataset'
files=['patient.csv','route.csv','time.csv']

In [4]:
#kaggle.api.dataset_download_files('sudalairajkumar/novel-corona-virus-2019-dataset')

In [5]:
#for file in files:
#    kaggle.api.dataset_download_file(kg_url,file,force=True, path='data')#force =Trure rewrites file if excist

In [ ]:
df_pat = pd.read_csv('data/'+files[0])
df_route = pd.read_csv('data/'+files[1])
df_time = pd.read_csv('data/'+files[2])
#df_time =df_time.reset_index()

In [ ]:
df_patient = pd.read_csv('data/patient.csv')

In [ ]:
df_patient.info()

In [ ]:
#df_time.rename(columns={'176':'Date'},inplace=True)
df_time['date']=pd.to_datetime(df_time['date'])

In [ ]:
df_patient.info()

In [ ]:
date_clm = ['confirmed_date','released_date','deceased_date']
for clm in date_clm:
    df_patient[clm]=pd.to_datetime(df_patient[clm])

In [ ]:
df_patient['duration']=df_patient['released_date']-df_patient['confirmed_date']
df_patient['duration_death']=df_patient['deceased_date']-df_patient['confirmed_date']
df_patient['age'] =2019- df_patient['birth_year']

In [ ]:
df_patient['age'].hist(bins=20)
plt.xlabel('age')

In [ ]:
df_patient['duration'].astype('timedelta64[D]').plot.hist(label='Recovered')
df_patient['duration_death'].astype('timedelta64[D]').plot.hist(label='Deseased')
plt.xlabel('days')
plt.legend()
plt.title('Duration of disease')
plt.show()

In [ ]:
df_patient[df_patient['state']=='deceased']['age'].hist(bins=20,alpha=0.5,label='decesed')
df_patient[df_patient['state']=='released']['age'].hist(bins=20,alpha=0.5,label='released')
plt.xlabel('age')
plt.title('Patient age distribution')
plt.legend();

In [ ]:
df_time=df_time.set_index('date')


In [ ]:
df_time['test'].plot(label='Tested')
df_time['confirmed'].plot(label='Confirmed')
df_time['released'].plot(label='Released')
plt.legend()


In [ ]:
import plotly.express as px

In [ ]:
df_time.head()

In [ ]:
fig = px.line(title='Cases')
fig.add_scatter(x=df_time.reset_index()['date'], y=df_time.reset_index()['confirmed'], mode='lines',name='Confiermed')
fig.add_scatter(x=df_time.reset_index()['date'], y=df_time.reset_index()['released'], mode='lines', name="Released")
fig.add_scatter(x=df_time.reset_index()['date'], y=df_time.reset_index()['deceased'], mode='lines', name="Deceased")
fig.show()

In [ ]:
fig = px.histogram(df_patient, x='age', range_x=[0,100], range_y=[0,30])
fig.show()

In [ ]:
df_patient.info()

## Model


In [5]:
df_p =pd.read_csv('data/patient.csv')
df_p.drop(columns=['patient_id','country','region',
                  'infected_by','infection_reason',
                   'infection_order','contact_number','group',
                  'disease'],inplace=True)
date_list =['confirmed_date','released_date','deceased_date']
for name in date_list:
    df_p[name]=pd.to_datetime(df_p[name])
df_p['sex'].fillna('Unkown', inplace=True)

In [6]:
df_p['recovery_length']=df_p['released_date']-df_p['confirmed_date']
df_p['before_deceased_days']=df_p['deceased_date']-df_p['confirmed_date']

In [7]:
df_p.head()

,sex,birth_year,confirmed_date,released_date,deceased_date,state,recovery_length,before_deceased_days
0,female,1984.0,2020-01-20,2020-02-06,NaT,released,17 days,NaT
1,male,1964.0,2020-01-24,2020-02-05,NaT,released,12 days,NaT
2,male,1966.0,2020-01-26,2020-02-12,NaT,released,17 days,NaT
3,male,1964.0,2020-01-27,2020-02-09,NaT,released,13 days,NaT
4,male,1987.0,2020-01-30,2020-03-02,NaT,released,32 days,NaT


In [8]:
df_for_model=df_p[['sex','birth_year','state']]
df_for_model.shape


(7869, 3)

In [9]:
df_for_model = df_for_model[df_for_model['birth_year'].notna()]
df_for_model.fillna( pd.Timedelta(seconds=0), inplace=True)
#df_for_model['recovery_length']=df_for_model['recovery_length'].dt.days.astype('int16')
#df_for_model['before_deceased_days']=df_for_model['before_deceased_days'].dt.days.astype('int16')

In [10]:
df_for_model=df_for_model[(df_for_model['state']=='deceased') | (df_for_model['state']=='released' )]
df_for_model.birth_year  = df_for_model.birth_year.astype('int16')


In [11]:
df_for_model

,sex,birth_year,state
0,female,1984,released
1,male,1964,released
2,male,1966,released
3,male,1964,released
4,male,1987,released
...,...,...,...
5459,male,1940,deceased
5766,male,1938,deceased
5767,female,1935,deceased
6284,female,1929,deceased


In [30]:
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import xgboost
from sklearn.model_selection import train_test_split as tts
import sklearn
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import pickle

In [13]:

df_hot_encoded = pd.get_dummies(df_for_model, drop_first=True) 

In [14]:
df_hot_encoded.head()

,birth_year,sex_male,state_released
0,1984,0,1
1,1964,1,1
2,1966,1,1
3,1964,1,1
4,1987,1,1


In [15]:
X = df_hot_encoded.drop('state_released',axis=1)
Y = df_hot_encoded['state_released']

In [16]:
X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.2)

In [17]:
sm = SMOTE(sampling_strategy='minority', random_state=7)
oversampled_trainX, oversampled_trainY = sm.fit_sample(X_train, y_train)


In [18]:
xgb= XGBClassifier()

In [19]:
xgb.fit(oversampled_trainX,oversampled_trainY)
xgb.score(X_test, y_test)

0.8421052631578947

In [20]:
xgb.fit(X_train,y_train)
xgb.score(X_test, y_test)

0.8947368421052632

In [21]:
rfc = RandomForestClassifier()
rfc.fit(oversampled_trainX,oversampled_trainY)
rfc.score(X_test, y_test)

0.8421052631578947

In [31]:
rfr = RandomForestRegressor()
rfr.fit(oversampled_trainX,oversampled_trainY)
rfr.score(X_test, y_test)

0.38623928356481485

In [44]:
for year in range(1944,1978):
    df_test = pd.DataFrame(data={'birth_year':[year],'sex_male':[0] })
    print(year,rfr.predict(df_test)[0])

1944 0.74
1945 0.58
1946 0.56
1947 0.56
1948 0.17
1949 0.01
1950 0.01
1951 0.01
1952 0.01
1953 0.01
1954 0.01
1955 0.3
1956 0.51
1957 0.76
1958 0.79
1959 0.98
1960 0.9416666666666668
1961 0.7601666666666667
1962 0.48224999999999996
1963 0.12374999999999999
1964 0.9183333333333333
1965 0.9783333333333333
1966 0.9783333333333333
1967 1.0
1968 1.0
1969 1.0
1970 1.0
1971 1.0
1972 1.0
1973 1.0
1974 1.0
1975 0.9866666666666666
1976 0.9866666666666666
1977 0.9528333333333333


In [34]:
xgb.predict(pd.DataFrame(data={'birth_year':[1988],'sex_male':[1] }))[0]

1

0.0

In [27]:
pickle_name='model_RFC_sklearn_V_'+sklearn.__version__.replace('.','_')+'.pickle'
pickle_out = open(pickle_name, 'wb')
pickle.dump(rfc, pickle_out)
pickle_out.close()


In [57]:
X.shape

(92, 2)

In [80]:
pickle_name='model_xgboost_V_'+xgboost.__version__.replace('.','_')+'.pickle'
pickle_out = open(pickle_name, 'wb')
pickle.dump(xgb, pickle_out)
pickle_out.close()